IBM CAPSTONE PROJECT 
In this project I will find out the best neighbourhood for potential Ethiopian/Eritrean restaurant 

In [4]:
#get the required libraries 
import pandas as pd 
import requests 
from bs4 import BeautifulSoup

In [5]:
#process and covert the html data
link=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") 
soup = BeautifulSoup(link.content,'lxml')

In [6]:
toronto_table=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    #ignore cells with a borough that is not assigned
    if row.span.text=='Not assigned':
        pass
    #If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        toronto_table.append(cell)

# Dataframe with 3 columns
df_toronto1=pd.DataFrame(toronto_table)
df_toronto1['Borough']=df_toronto1['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df_toronto1

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
#get the coordinates
url='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_geo1=pd.read_csv(url)
df_geo1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
#rename column heading 
df_geo1=df_geo1.rename(columns = {'Postal Code': 'PostalCode'})
df_geo1.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df_toronto1 = pd.merge(df_toronto1, df_geo1, on = 'PostalCode')
df_toronto1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [10]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=2155
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2155
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37ha70c682_1
done

# All requested packages already installed.

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=2155
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2155
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37ha70c682_1
done

# All requested packages already installed.



In [11]:
#show boroughs containing 'Toronto' in the title
df_toronto2=df_toronto1[df_toronto1['Borough'].str.contains('Toronto')]
df_toronto3=df_toronto2.reset_index(drop=True)
df_toronto3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [13]:
df_toronto3['Borough'].value_counts()

Downtown Toronto          17
Central Toronto            9
West Toronto               6
East Toronto               4
East York/East Toronto     1
East Toronto Business      1
Downtown Toronto Stn A     1
Name: Borough, dtype: int64

In [15]:
# Create a new column as Label with integer value
df_toronto3['Label']=df_toronto3['Borough'].replace(to_replace=['Downtown Toronto','Central Toronto','West Toronto','East Toronto', 'East Toronto Business', 'East York/East Toronto', 'Downtown Toronto Stn A'],value=[1,2,3,4,5,6,7],inplace=False)
df_toronto3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Label
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,4
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1


In [16]:
#use geolocator to find toronto coordinates
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinates of Toronto are {latitude}, {longitude}.')

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto3['Latitude'], 
                                           df_toronto3['Longitude'],
                                           df_toronto3['Borough'], 
                                           df_toronto3['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#31866cc',
        fill_opacity=0.8).add_to(map_toronto)  
    
map_toronto

In [18]:
#Now we will define 4square credentials 
CLIENT_ID = '30BSUGHBDF4SFUO3FRUAYQBVJS0CZV00FD502U0YP5ANTEYB' # your Foursquare ID
CLIENT_SECRET = 'BEFPUSYTWMS0PD42V0BW0LIVE1NNAFF210GMPCUERNXAT231' # your Foursquare Secret
VERSION = 20200514 # Foursquare API version

print('Credentials Stored')

Credentials Stored


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100 # limit of number of venues returned by Foursquare API
    radius = 500 # define radius
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=df_toronto3['Neighborhood'],
                                latitudes=df_toronto3['Latitude'],
                                longitudes=df_toronto3['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [21]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [23]:
#get number of venues per neighbourhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,64,64,64,64,64,64
Christie,15,15,15,15,15,15
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,37,37,37,37,37,37
Davisville North,8,8,8,8,8,8


In [24]:
#number of unique venues 
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 231 uniques categories.


In [25]:
print("The Venue Categories are", toronto_venues['Venue Category'].unique())

The Venue Categories are ['Bakery' 'Coffee Shop' 'Distribution Center' 'Spa' 'Restaurant' 'Park'
 'Breakfast Spot' 'Gym / Fitness Center' 'Historic Site' 'Chocolate Shop'
 'Farmers Market' 'Pub' 'Performing Arts Venue' 'Dessert Shop'
 'French Restaurant' 'Yoga Studio' 'Café' 'Theater' 'Event Space'
 'Shoe Store' 'Art Gallery' 'Brewery' 'Cosmetics Shop' 'Electronics Store'
 'Bank' 'Beer Store' 'Hotel' 'Wine Shop' 'Antique Shop' 'Clothing Store'
 'Burrito Place' 'Pizza Place' 'Comic Shop' 'Plaza' 'Music Venue'
 'Ramen Restaurant' 'Sporting Goods Shop' 'Sandwich Place'
 'Mexican Restaurant' 'Japanese Restaurant' 'Gastropub' 'Diner'
 'Middle Eastern Restaurant' 'Tanning Salon' 'Shopping Mall' 'Steakhouse'
 'Fast Food Restaurant' 'Burger Joint' 'College Rec Center'
 'Thai Restaurant' 'Modern European Restaurant' 'New American Restaurant'
 'Sushi Restaurant' 'Tea Room' 'Lake' 'Movie Theater' 'Lounge' 'Bookstore'
 'Furniture / Home Store' 'Ethiopian Restaurant' 'Seafood Restaurant'
 'Chinese 

In [26]:
#Is there any eritrean restaurants?
"Eritrean Restaurant" in toronto_venues['Venue Category'].unique()

False

In [27]:
#Is there any ethiopian restaurants?
"Ethiopian Restaurant" in toronto_venues['Venue Category'].unique()

True

In [28]:
#Is there any habesha restaurants?
"Habesha Restaurant" in toronto_venues['Venue Category'].unique()

False

In [30]:
#Now analyze each neighbourhood individually 
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1583, 232)


,Neighborhoods,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
#group rows by neighbourhoods and the mean of occurrence for each category
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped.head()

(39, 232)


,Neighborhoods,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016949,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.015625,0.0,0.0,0.015625,0.0,0.015625
4,Christie,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000


In [32]:
num_top_venues = 5

for hood in to_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = to_grouped[to_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.10
1  Cocktail Bar  0.07
2        Bakery  0.05
3    Restaurant  0.03
4   Cheese Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3                 Gym  0.05
4  Italian Restaurant  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4           Airport  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.19
1      Sandwich Place  0.06
2                Café  0.05
3  Italian Restaurant  0.05
4        Burger Joint  0.03


----Christie----
           venue  freq
0  Grocery Store  0.27
1           Café  0.20
2           Park  0.13
3      Nightclub  0.07
4     Restaurant  0.0

In [51]:
eth = to_grouped[["Neighborhoods","Ethiopian Restaurant"]]
eth 

,Neighborhoods,Ethiopian Restaurant
0,Berczy Park,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
3,Central Bay Street,0.000000
4,Christie,0.000000
5,Church and Wellesley,0.012821
6,"Commerce Court, Victoria Hotel",0.000000
7,Davisville,0.000000
8,Davisville North,0.000000
9,"Dufferin, Dovercourt Village",0.000000


In [35]:
def return_most_common_venues(row, num_top_venues):    
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = to_grouped['Neighborhoods']

for ind in np.arange(to_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Cheese Shop,Seafood Restaurant,Pharmacy,Farmers Market,Beer Bar,Concert Hall
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Gym,Italian Restaurant,Stadium,Nightclub,Furniture / Home Store,Bar,Bakery
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Plane,Airport,Bar,Coffee Shop,Boat or Ferry,Rental Car Location,Boutique
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burger Joint,Salad Place,Bubble Tea Shop,Business Service,Restaurant,Donut Shop
4,Christie,Grocery Store,Café,Park,Nightclub,Restaurant,Baby Store,Italian Restaurant,Coffee Shop,Candy Store,Monument / Landmark


In [38]:
neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Cheese Shop,Seafood Restaurant,Pharmacy,Farmers Market,Beer Bar,Concert Hall
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Gym,Italian Restaurant,Stadium,Nightclub,Furniture / Home Store,Bar,Bakery
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Plane,Airport,Bar,Coffee Shop,Boat or Ferry,Rental Car Location,Boutique
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burger Joint,Salad Place,Bubble Tea Shop,Business Service,Restaurant,Donut Shop
4,Christie,Grocery Store,Café,Park,Nightclub,Restaurant,Baby Store,Italian Restaurant,Coffee Shop,Candy Store,Monument / Landmark


In [1]:
from sklearn.cluster import KMeans

In [39]:
# set number of clusters to 5 clusters
kclusters = 5

to_grouped_clustering = to_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(to_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [40]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

to_merged = toronto_venues

# merge to_grouped with toronto_venues to add latitude/longitude for each neighborhood
to_merged = to_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhood')

In [41]:
to_merged

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,0,Coffee Shop,Pub,Bakery,Café,Park,Breakfast Spot,Theater,Brewery,Spa,Shoe Store
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,0,Coffee Shop,Pub,Bakery,Café,Park,Breakfast Spot,Theater,Brewery,Spa,Shoe Store
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,0,Coffee Shop,Pub,Bakery,Café,Park,Breakfast Spot,Theater,Brewery,Spa,Shoe Store
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa,0,Coffee Shop,Pub,Bakery,Café,Park,Breakfast Spot,Theater,Brewery,Spa,Shoe Store
4,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant,0,Coffee Shop,Pub,Bakery,Café,Park,Breakfast Spot,Theater,Brewery,Spa,Shoe Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578,Enclave of M4L,43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station,0,Light Rail Station,Comic Shop,Auto Workshop,Park,Restaurant,Skate Park,Burrito Place,Farmers Market,Fast Food Restaurant,Brewery
1579,Enclave of M4L,43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park,0,Light Rail Station,Comic Shop,Auto Workshop,Park,Restaurant,Skate Park,Burrito Place,Farmers Market,Fast Food Restaurant,Brewery
1580,Enclave of M4L,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa,0,Light Rail Station,Comic Shop,Auto Workshop,Park,Restaurant,Skate Park,Burrito Place,Farmers Market,Fast Food Restaurant,Brewery
1581,Enclave of M4L,43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station,0,Light Rail Station,Comic Shop,Auto Workshop,Park,Restaurant,Skate Park,Burrito Place,Farmers Market,Fast Food Restaurant,Brewery


In [50]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Latitude'], to_merged['Longitude'], to_merged['Neighbourhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi)+ ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Latitude'

In [48]:
#Cluster1
to_merged.loc[to_merged['Cluster Labels'] == 0, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.654260,-79.362017,Bakery,0,Coffee Shop,Café,Park,Bakery,Breakfast Spot,Theater,Pub,Farmers Market,Brewery,Chocolate Shop
1,43.654260,-79.361809,Coffee Shop,0,Coffee Shop,Café,Park,Bakery,Breakfast Spot,Theater,Pub,Farmers Market,Brewery,Chocolate Shop
2,43.654260,-79.358008,Distribution Center,0,Coffee Shop,Café,Park,Bakery,Breakfast Spot,Theater,Pub,Farmers Market,Brewery,Chocolate Shop
3,43.654260,-79.359874,Spa,0,Coffee Shop,Café,Park,Bakery,Breakfast Spot,Theater,Pub,Farmers Market,Brewery,Chocolate Shop
4,43.654260,-79.356211,Park,0,Coffee Shop,Café,Park,Bakery,Breakfast Spot,Theater,Pub,Farmers Market,Brewery,Chocolate Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577,43.662744,-79.319898,Park,0,Light Rail Station,Gym / Fitness Center,Restaurant,Pizza Place,Farmers Market,Smoke Shop,Burrito Place,Skate Park,Fast Food Restaurant,Auto Workshop
1578,43.662744,-79.325145,Light Rail Station,0,Light Rail Station,Gym / Fitness Center,Restaurant,Pizza Place,Farmers Market,Smoke Shop,Burrito Place,Skate Park,Fast Food Restaurant,Auto Workshop
1579,43.662744,-79.325379,Smoke Shop,0,Light Rail Station,Gym / Fitness Center,Restaurant,Pizza Place,Farmers Market,Smoke Shop,Burrito Place,Skate Park,Fast Food Restaurant,Auto Workshop
1580,43.662744,-79.326911,Gym / Fitness Center,0,Light Rail Station,Gym / Fitness Center,Restaurant,Pizza Place,Farmers Market,Smoke Shop,Burrito Place,Skate Park,Fast Food Restaurant,Auto Workshop


In [49]:
#cluster 2
to_merged.loc[to_merged['Cluster Labels'] == 1, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
959,43.696948,-79.406873,Trail,1,Park,Sushi Restaurant,Trail,Jewelry Store,Adult Boutique,Movie Theater,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant
960,43.696948,-79.406605,Park,1,Park,Sushi Restaurant,Trail,Jewelry Store,Adult Boutique,Movie Theater,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant
961,43.696948,-79.407957,Sushi Restaurant,1,Park,Sushi Restaurant,Trail,Jewelry Store,Adult Boutique,Movie Theater,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant
962,43.696948,-79.407644,Jewelry Store,1,Park,Sushi Restaurant,Trail,Jewelry Store,Adult Boutique,Movie Theater,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant
1147,43.689574,-79.383438,Park,1,Playground,Park,Trail,Tennis Court,Men's Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Music Venue,Modern European Restaurant
1148,43.689574,-79.386841,Trail,1,Playground,Park,Trail,Tennis Court,Men's Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Music Venue,Modern European Restaurant
1149,43.689574,-79.382900,Tennis Court,1,Playground,Park,Trail,Tennis Court,Men's Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Music Venue,Modern European Restaurant
1150,43.689574,-79.383465,Playground,1,Playground,Park,Trail,Tennis Court,Men's Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Music Venue,Modern European Restaurant
1243,43.679563,-79.378934,Playground,1,Park,Playground,Trail,Music Venue,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
1244,43.679563,-79.373788,Park,1,Park,Playground,Trail,Music Venue,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


In [50]:
#Cluster3
to_merged.loc[to_merged['Cluster Labels'] == 2, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
951,43.711695,-79.420702,Home Service,2,Home Service,Garden,Adult Boutique,Neighborhood,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant
952,43.711695,-79.411978,Garden,2,Home Service,Garden,Adult Boutique,Neighborhood,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant


In [51]:
#Cluster4
to_merged.loc[to_merged['Cluster Labels'] == 3, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
230,43.676357,-79.293942,Trail,3,Neighborhood,Trail,Health Food Store,Pub,Museum,Movie Theater,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Adult Boutique
231,43.676357,-79.297734,Health Food Store,3,Neighborhood,Trail,Health Food Store,Pub,Museum,Movie Theater,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Adult Boutique
232,43.676357,-79.297215,Pub,3,Neighborhood,Trail,Health Food Store,Pub,Museum,Movie Theater,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Adult Boutique
233,43.676357,-79.292869,Neighborhood,3,Neighborhood,Trail,Health Food Store,Pub,Museum,Movie Theater,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Adult Boutique


In [52]:
#Cluster4
to_merged.loc[to_merged['Cluster Labels'] == 3, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
230,43.676357,-79.293942,Trail,3,Neighborhood,Trail,Health Food Store,Pub,Museum,Movie Theater,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Adult Boutique
231,43.676357,-79.297734,Health Food Store,3,Neighborhood,Trail,Health Food Store,Pub,Museum,Movie Theater,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Adult Boutique
232,43.676357,-79.297215,Pub,3,Neighborhood,Trail,Health Food Store,Pub,Museum,Movie Theater,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Adult Boutique
233,43.676357,-79.292869,Neighborhood,3,Neighborhood,Trail,Health Food Store,Pub,Museum,Movie Theater,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Adult Boutique


In [52]:
#Cluster5
to_merged.loc[to_merged['Cluster Labels'] == 4, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
945,43.72802,-79.394382,Park,4,Park,Bus Line,Swim School,Dim Sum Restaurant,Adult Boutique,Music Venue,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
946,43.72802,-79.394260,Dim Sum Restaurant,4,Park,Bus Line,Swim School,Dim Sum Restaurant,Adult Boutique,Music Venue,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
947,43.72802,-79.382860,Swim School,4,Park,Bus Line,Swim School,Dim Sum Restaurant,Adult Boutique,Music Venue,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
948,43.72802,-79.382805,Bus Line,4,Park,Bus Line,Swim School,Dim Sum Restaurant,Adult Boutique,Music Venue,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
